In [ ]:
import os
import pandas as pd
import numpy as np
import finlab
from finlab import data, backtest
import _beowFmt as fm 

current_path = os.getcwd()
key = open(f"{current_path}/config.txt", "r").read()
finlab.login(key)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 忽略FutureWarning


close = data.get('price:收盤價')
vol = data.get('price:成交股數')
sma20 = close.average(20)

# 近一日收盤價創近60日新高
closeOver60Days = close.rolling(60, min_periods=1).max() == close
c1 = closeOver60Days

# 前30日(不包含今日)有至少一日的收盤價未創新高
c2 = (closeOver60Days.shift(1) == 0).rolling(30).sum() > 0

# 第30日～第55日前的收盤價有至少一日收盤價創近60日新高
c3 = (closeOver60Days.shift(30).rolling(25).sum() > 0)

# 前30日～55日的收盤價最高價小於近日收盤價
c4 = (close.shift(30).rolling(25).max() < close)

# 近日收盤價大於120前收盤價
c5 = close > close.shift(120)

# 近日收盤價大於60前收盤價
c6 = close > close.shift(60)

rev = data.get('monthly_revenue:當月營收')
#print(rev)

# 近3月平均營收大於12月平均營收
c7 = rev.average(3) > rev.average(12)

# 近5日平均成交量大於近20日平均成交量
c8 = vol.average(5) > vol.average(20)

# 近5日平均成交量 > 2000
c9 = vol.average(5)/1000 > 2000

# 買進策略
buy = c1 & c2 & c3 & c4 & c5 & c6 & c7 & c8 & c9
# 月線下彎
sell = sma20 < sma20.shift()

position = pd.DataFrame(np.nan, index=buy.index, columns=buy.columns)
position[buy] = 1
position[sell] = 0
position = position.ffill().fillna(0)


report = backtest.sim(position.loc['2014':], resample="D", name='二次創高股票', live_performance_start='2018-06-01')

fm.trans2CSVfile(position, 'data/xq_import_二次創高/二次創高')


In [ ]:
import os
import numpy as np
from finlab import data, backtest

current_path = os.path.dirname(os.getcwd())
import finlab
finlab.login(open(f"{current_path}/config.txt", "r").read())

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 忽略FutureWarning

close =  data.get('price:收盤價')
volume = data.get('price:成交股數')
revenue = data.get('monthly_revenue:當月營收')

print(revenue["5251"])